# 类元编程

类元编程：在程序运行时创建或者定制类的编程手段

Python中，类是一等对象，因此Python能够在任何时候使用函数新建类。Python的类装饰器是函数，类装饰器能够审查、修改甚至替换现有的类。

## 类工厂函数

与其他工厂函数一样，对于类的创建，我们希望能有一个工厂函数能够创建具有指定属性以及方法的类，而不用写冗长的样板代码。

一个简单的类工厂函数的核心是type类，type类能够依照传入的类名（name）、父类（bases）以及存有属性键值对的字典（dict）创建一个新类。一个简单的类工厂函数如下

In [5]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError:
        pass

    # 根据传入的field_names创建记录有属性名称的元组
    field_names = tuple(field_names)

    def __init__(self, *args, **kwargs):

        # 按照位置参数的顺序为属性赋值
        attrs = dict(zip(self.__slots__, args))
        # 添加/更新关键字参数
        attrs.update(kwargs)
        # 进行参数值的设定
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self):
        # 将类的实例变为可迭代的对象
        for name in self.__slots__:
            yield getattr(self, name)
    
    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(
        __slots__ = field_names,
        __init__ = __init__,
        __iter__ = __iter__,
        __repr__ = __repr__
    )

    return type(cls_name, (object, ), cls_attrs)


# 测试代码

# 创建类
Dog = record_factory("Dog", "name weight owner")
# 创建类实例
rex = Dog("Rex", 30, "Bob")
rex

Dog(name='Rex', weight=30, owner='Bob')

上述record_factory的关键是：解析传入的field_names并且将相关属性记入\_\_slots\_\_，此外在\_\_init\_\_中解析传入的参数也是非常关键的步骤。

## 定制描述符的类装饰器

这里以定制描述符来介绍类装饰器。具体而言，为了不显式的指定描述符中存储的被托管属性的名称，上一章采取了利用计数器自动累计创建属性的方法。这种方法虽然能够实现不显式指定描述符中存储的被托管属性的名称的问题，但是没有明确指向的名称为debug带来不便。毫无疑问，上一章中用于属性检验的描述符最好是既不用显式指定名称，又能自行生成包含被描述符接管的属性名称的存储属性。

由于实例化描述符时无法得到被托管属性的名称，因此上述的最佳方式无法实现。但在本章中，由于类装饰器作用于整个类因此类装饰器能够审查整个类，这使得为描述符设置合理的存储属性名称成为可能。

具体而言，核心流程是：

1. 在类装饰器中遍历类属性并寻找是指定描述符实例的属性
2. 修改该属性中存储属性的名称

由于在类装饰器审查整个类时，类已经完成创建，因此能够非常简单的获得被描述符接管读写操作的属性的名称。一个实现上述功能的类装饰器如下所示

In [ ]:
def entity(cls):
    # 遍历类属性
    for key, attr in cls.__dict__.items():
        # 查找为描述符的属性
        if isinstance(attr, Validated):
            # 替换存储属性的名称
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
    
    # 返回修改后的类
    return cls

类装饰器能够很轻松的实现定制类的功能。但是类装饰器的缺陷是：被装饰的类的子类可能继承或不继承类装饰器。这一缺陷在函数装饰器中不会出现，这也是类的特性导致的问题

## Python的导入时和运行时

导入时：
解释器会从上到下一次性解析完整个.py的源码并生成字节码，句法错误就是在此时抛出。

值得注意的是import模块时执行的操作。Python中的import不只是声明，import时很可能运行一些用户代码，这也是为什么要用 if \_\_name\_\_ == "\_\_main\_\_"避免一些代码在模块导入时被意外运行的原因。

对于模块中的函数，解释器会处理函数的定义体并将函数对象绑定到对应的全局名称上，此时解释器并不会执行函数

对于模块中的类，解释器会执行类的定义体并定义类的属性和方法，然后构建类对象。

从上述描述可以发现，Python的导入时和运行时的界限相对模糊，导入时也会执行一些看起来应当在运行时才执行的操作


本书通过一个实例来展示上述特点。该实例可以总结得到如下的结论：
1. 在导入时，函数会被处理，但是解释器不会执行函数的定义体
2. 在导入时，类的定义体会被执行
3. 基于上一条，甚至嵌套的类也会被执行
4. 对于被类装饰器装饰的类，类的定义体会首先被执行，然后才运行装饰器函数
5. 在导入时，if \_\_name\_\_ == "\_\_main\_\_"中的语句不会被执行
6. 类装饰器一般情况下对子类没有影响，除非该子类使用super()调用父类中被类装饰器处理过的方法

## 元类基础

Python中类是对象，因此类肯定也是另外某个类的实例（比较绕）。下述例子描述了这一点

In [9]:
print(str.__class__)
print(list.__class__)
print(type.__class__)
print(object.__class__)

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


上述例子中，内置的str和list均具有属性\_\_class\_\_，而且该属性的值均为type类；此外，type类的\_\_class\_\_属性也为type；最神奇的是，object居然也是type的实例。即Python的类是type类的实例，而type类则是其自身的实例。注意上述描述的类和实例之间的关系，而不是父类和子类之间的关系。下述例子描述了父类以及子类

In [7]:
print(str.__mro__)
print(type.__mro__)
print(object.__mro__)

(<class 'str'>, <class 'object'>)
(<class 'type'>, <class 'object'>)
(<class 'object'>,)


上述例子表明，所有其他类均是object的子类，type类也不例外。

根据上述测试有如下总结：

1. 所有类均是type类的实例
2. 所有其他类均是object的子类
3. type类是其自身的实例
4. object类是type的实例，type是object的子类

除了type外，Python还有其他一些元类。例如抽象基类collections.Iterable是abc.ABCMeta的实例。（抽象基类是其他类的实例，这个关系还是挺魔幻的）

元类和类装饰器一样，能够定制类，除此之外，元类更强大：
1. 继承自特点元类的类的\_\_init\_\_会被该元类接管，类装饰器则是在类创建完毕后定制类
2. 无论一个特定的类是否直接继承自特定元类，只要父类继承自该元类，该类就会受到特定元类的影响；类装饰器对子类的影响有条件

## 定制描述符的元类

类似于类装饰器，元类也能够实现定制描述符的功能。其代码如下，实际功能部分和类装饰器中的实现方法类似，结合上一章的LineItem以及Validated类可以实现设值验证功能。为了方便测试，所有代码展示如下

In [25]:
import abc


# -----------------
# 描述符模板方法
# -----------------

class AutoStorage:
    """
    最高层次的抽象
    仅实现了自动存储和管理属性的必要属性以及接口
    """

    _counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls._counter
        self.storage_name = "_{}#{}".format(prefix, index)
        cls._counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)


class Validated(abc.ABC, AutoStorage):
    """
    低一层次的抽象
    为那些需要检验设值的类提供统一接口
    """
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    
    @abc.abstractmethod
    def validate(self, instance, value):
        """
        return validated value or rasie ValueError
        """


class Quantity(Validated):
    """
    Validated的具体子类
    用于检验设值是否大于0
    """

    def validate(self, instance, value):
        if value <= 0:
            raise ValueError("value must be > 0")
        return value


# -----------------
# 元类
# -----------------

class EntityMeta(type):
    """
    实现定制描述符功能的元类
    """
    def __init__(cls, name, bases, attr_dict):

        # 相当于调用type(name, bases, attr_dict)
        # 创建类
        super().__init__(name, bases, attr_dict)

        # 定制描述符
        # 和类装饰器中的实现一致
        # 遍历类属性
        for key, attr in cls.__dict__.items():
            # 查找为描述符的属性
            if isinstance(attr, Validated):
                # 替换存储属性的名称
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)


class Entity(metaclass = EntityMeta):
    """实现定制描述符功能的类"""


# 一个简单的购物类
# class LineItem(metaclass = EntityMeta):
class LineItem(Entity):
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price


test_item1 = LineItem("test1", 10, 1)
print(test_item1.weight)
print(getattr(test_item1, '_Quantity#weight'))
print(LineItem.weight.storage_name)
print(dir(test_item1)[:2])

10
10
_Quantity#weight
['_Quantity#price', '_Quantity#weight']


## \_\_prepare\_\_方法

默认情况下attr_dict是一个字典，因此属性的顺序丢失了。Python3后，元类新增了\_\_prepare\_\_方法。\_\_prepare\_\_方法要求使用@classmethod装饰器定义并且该方法会在\_\_new\_\_之前被调用，其返回值为一个映射，该映射用于接受attr_dict。

上述描述比较抽象，简单来说，\_\_prepare\_\_可以返回一个容器用于存储类属性，此外，\_\_prepare\_\_返回的容器还可以事先记录一些键值对，这些键值对会成为类属性。

示例如下

In [34]:
import collections


class EntityMeta(type):
    """元类"""

    @classmethod
    def __prepare__(cls, name, bases):
        _temp = collections.OrderedDict()
        _temp["temp"] = 10

        return _temp
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)

class Entity(metaclass=EntityMeta):
    """"""


class TestClass(Entity):

    def __init__(self):
        pass


for name, value in TestClass.__dict__.items():
    print(name, value)

temp 10
__module__ __main__
__init__ <function TestClass.__init__ at 0x000002339A5678B8>
__doc__ None


上述示例中，\_\_prepare\_\_增加的属性出现在了TestClass的类属性中

## 总结

1. 类元编程应该读成类·元编程，即动态创建或者定制类的编程手段
2. Python中类是一等对象，这意味这类是其他某些东西的实例。事实上，所有的类均是type类的实例，type类是其自身的实例
3. 类装饰器是作用于类的装饰器，这类装饰器可以实现定制类
4. 类装饰器在类创建完毕后生效，并且类装饰器不一定会作用于被其装饰的类的子类
5. Python的import不仅是一种声明，在import时，解释器会处理但不执行所有的函数、执行所有的类以及嵌套类的定义体并且执行没有被if \_\_name\_\_ == "\_\_main\_\_"语句作用的用户代码
6. type不仅能用来获取类的类型，还能用来创建类。具体来说，type会根据传入的类名、父类以及存有属性键值对的字典创建一个新类
7. 元类会接管其子类的\_\_init\_\_方法，用来创建类
8. 元类相较于类装饰器要更强大，其会作用于所有直接继承或者间接继承自该类的子类
9. 上述仅是类元编程初步
10. 与抽象基类类似，大多数情况下无需定义元类